This notebook will demonstrate how to train a convolutional neural network model to infer surface water extent from a stack of images containing optical, SAR, and elevation information

In [ ]:
# ML imports
from model import sarDataLoader, sarInferenceModel
import albumentations
import torch

# data tools imports
import pandas as pd

# misc imports
import os
import random
from pathlib import Path

random.seed(42) # for repeatability
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Force kernel to use this GPU

In [ ]:
# The paths to training data are loaded, with 90% used for training and 10% used for validation
df = pd.read_csv('../data/training_data/chips/training_data.csv')
train = df.sample(frac=0.9)
test = df.drop(train.index)

In [ ]:
# To prevent the model from overfitting to the specific shapes of water bodies contained in training chips, we apply rotation and flip transformations
data_transforms = albumentations.Compose([
    albumentations.VerticalFlip(),
    albumentations.HorizontalFlip(),
    albumentations.RandomRotate90()
])

In [ ]:
# We first train a model that uses only SAR data to make inferences
model_params = {
    "sarData":True,
    "denoisingWeight":.35,
    "opticalData":False,
    "demData":True,
    "handData":True,
    "output_classes":2,
    "class_weights": [5., 1.],
    "channel_drop":True,
    "backbone" : "resnet50",
    "weights": "imagenet",
    "patience": 80,
    "batch_size": 32,
    "num_workers": 8,
    "lossfunction": None,
    "lr": 5e-5,
    "min_epochs": 1,
    "max_epochs": 300,
    "num_sanity_val_steps":0,
    "gpu": True,
    "ngpus":1,
    "output_path": "model-outputs",
    "log_path": "tensorboard_logs",
    "experiment_name" : "sar_only_model"
}

training_data = {'data':train.drop(['labels'], axis=1), 'labels': train[['labels']], "transforms":data_transforms}
val_data = {'data':test.drop(['labels'], axis=1), 'labels': test[['labels']]}

# Instantiate model
model = sarInferenceModel(model_params, sarDataLoader, training_data, val_data)  # The loss function instantiation assumes that all non-labeled pixels are -1

model.fit()

model_save_path = Path.cwd() / model_params.get('output_path', 'model_save_dir')
model_save_path.mkdir(exist_ok=True)
model_path = model_save_path / Path(model_params.get("experiment_name")).with_suffix(".pt")
torch.save(model.state_dict(), model_path)

In [ ]:
# This model uses both optical and SAR data to make inferences
model_params = {
    "sarData":True,
    "denoisingWeight":.35,
    "opticalData":True,
    "demData":True,
    "handData":True,
    "output_classes":2,
    "class_weights": [5., 1.],
    "channel_drop":True,
    "backbone" : "resnet50",
    "weights": "imagenet",
    "patience": 80,
    "batch_size": 10,
    "num_workers": 8,
    "lossfunction": None,
    "lr": 5e-5,
    "min_epochs": 1,
    "max_epochs": 300,
    "num_sanity_val_steps":0,
    "gpu": True,
    "ngpus":1,
    "output_path": "model-outputs",
    "log_path": "tensorboard_logs",
    "experiment_name" : "sar_and_optical_model"
}

training_data = {'data':train.drop(['labels'], axis=1), 'labels': train[['labels']], "transforms":data_transforms}
val_data = {'data':test.drop(['labels'], axis=1), 'labels': test[['labels']]}

# Instantiate model
model = sarInferenceModel(model_params, sarDataLoader, training_data, val_data)  # The loss function instantiation assumes that all non-labeled pixels are -1

model.fit()

model_save_path = Path.cwd() / model_params.get('output_path', 'model_save_dir')
model_save_path.mkdir(exist_ok=True)
model_path = model_save_path / Path(model_params.get("experiment_name")).with_suffix(".pt")
torch.save(model.state_dict(), model_path)